In [2]:
# Psycopg is the most popular PostgreSQL database adapter for the Python programming language. 
!pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable


In [3]:
## import postgreSQL adapter for the Python
import psycopg2

#### Database creation from OS
jarminet@jarminet:~$ sudo -u postgres psql \
postgres=# CREATE DATABASE yourdbname; \
postgres=# CREATE USER youruser WITH ENCRYPTED PASSWORD 'yourpass'; \
postgres=# GRANT ALL PRIVILEGES ON DATABASE yourdbname TO youruser; \

#### Accesing to database from OS CLI
jarminet@jarminet:~$ psql -h localhost -p 5432 -d yourdbname -U youruser\

#### Creating table
yourdbname=> CREATE TABLE tabla1 (id INT PRIMARY KEY, name VARCHAR(50) NOT NULL);

#### Useful commands
\l  - List dabases \
\dt - List available tables \
\d table_name - Describe database \
\du - List all users and their assign roles

In [4]:
# Create connection to the database
conn = psycopg2.connect("host=127.0.0.1 dbname=yourdbname user=youruser password='yourpass'")

In [5]:
# Use the connection to get a cursor that will be used to execute queries.
cur = conn.cursor()

#### Auto-commit
Because we're working with an ACID database we need to make sure that we commit our transactions.  If we do not commit our transactions and we try to execute another action with the same open/broken transaction, we'll an erro in our transaction block.  We're blocked until we restart the connection or commit our transaction. \
Instead of worrying about committing after each transaction, let's just set auto-commit to TRUE.  This says that after each call during the session, commit htat one action and do not hold open the transaction for any other actions.

In [7]:
# set autocommit
conn.set_session(autocommit=True)

In [10]:
cur.execute("INSERT INTO tabla1 (id, name) VALUES (101, 'luis')")

In [11]:
cur.execute("INSERT INTO tabla1 (id, name) VALUES (102, 'Florian')")

In [19]:
cur.execute("SELECT * FROM tabla1;")
cur.fetchall()

[(101, 'luis'), (102, 'Florian')]

In [20]:
#Create another table
cur.execute("CREATE TABLE tabla2 (id INT PRIMARY KEY, profesion VARCHAR(50));")

In [21]:
# Insert values to the new table
cur.execute("INSERT INTO tabla2 (id, profesion) VALUES (201, 'Ingeniero')")

In [22]:
# Imprimir los datos
cur.execute("SELECT * FROM tabla2")
cur.fetchall()

[(201, 'Ingeniero')]

## Creating a table

In [28]:
#This try-except block is to make sure that errors are caught and understood.
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=yourdbname user=youruser password=yourpass")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [29]:
# Get a cursor
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

In [30]:
# With automactic ach action is commited without having to call conn.commit() after each command. 
# The ability to rollback and commit transactions are a feature of Relational Databases. 
conn.set_session(autocommit=True)

In [31]:
# The error is because the database udacity hasn't been created yet.
try: 
    cur.execute("select * from udacity.music_library")
except psycopg2.Error as e:
    print(e)

relation "udacity.music_library" does not exist
LINE 1: select * from udacity.music_library
                      ^



In [34]:
# Create the database
try: 
    cur.execute("CREATE DATABASE udacity")
except psycopg2.Error as e:
    print(e)

In [35]:
# Close our current connection and make a new one with udacity database

try: 
    conn.close()
except psycopg2.Error as e:
    print(e)
  
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=udacity user=youruser password=yourpass")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)

### We will create a Music Library of albums. Each album has a lot of information we could add to the music library table. We will  start with album name, artist name, year. 
`Table Name: music_library
column 1: Album Name
column 2: Artist Name
column 3: Year `
### Translate this information into a Create Table Statement. 

In [36]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS music_library(album_name VARCHAR, artist_name VARCHAR, year INT);")
except psycopg2.Error as e:
    print("Error: Could not create de database")
    print(e)

In [40]:
# Inserting values to Database
try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) VALUES (%s, %s, %s)", \
                ("Let It Be", "The Beatles", 1970))
except pyscopg2.Error as e:
    print("Error: Could not insert the values")
    print(e)

In [45]:
cur.execute("select * from music_library")
cur.fetchall()

[('Let It Be', 'The Beatles', 1970)]

In [46]:
try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) VALUES (%s, %s, %s)", \
               ("Abbey Road", "The beatles", 1969))
except pyscop2.Error as e:
    print("Error: Could not insert values to the table music_library")
    print(e)

In [49]:
cur.execute("SELECT * FROM music_library")
cur.fetchall()

[('Let It Be', 'The Beatles', 1970), ('Abbey Road', 'The beatles', 1969)]

In [51]:
cur.execute("SELECT * FROM music_library")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Let It Be', 'The Beatles', 1970)
('Abbey Road', 'The beatles', 1969)


In [52]:
# Close the cursor and connection.
cur.close()
conn.close()